In [1]:
from qkeras import *
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Softmax, Add, ZeroPadding2D, MaxPooling2D
from tensorflow.keras.backend import random_uniform as rand_tensor
import numpy as np
from qkeras import *
import math
from tensorflow import keras

c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
Widths = [10, 32, 60, 100]
Heights = [10, 32, 60, 100]
pSize = [(1,1), (2,2), (3,3)]
pStride = [(1,1), (1,2), (2,1), (2,2), (2,3), (3,2), (3,3)]
mode = ["same", "valid"]

# Widths = [60]
# Heights = [60]
# pSize = [(3,3)]
# pStride = [(2,2)]
# mode = ['same']

def myPooling2D(pool_type, y_arr, Pool_size, Stride, padding_type):
    assert len(Pool_size)==2 and len(Stride)==2, f"{len(Pool_size)}, {len(Stride)}"
    assert padding_type in {"same", "valid"}
    assert pool_type in {"max", "avg"}

    YN, YH, YW, YC = y_arr.shape
    PKH, PKW = Pool_size
    PSH, PSW = Stride

    if padding_type=="same":
        PXH = (YH+PSH-1)//PSH
        PXW = (YW+PSW-1)//PSW
    else:
        PXH = (YH-PKH+PSH)//PSH
        PXW = (YW-PKW+PSW)//PSW


    x_arr = np.zeros((YN, PXH, PXW, YC))

    p_st, q_st = 0, 0
    if padding_type == "same":
        p_st = max((PSH*(PXH-1)+PKH-YH)//2, 0)
        q_st = max((PSW*(PXW-1)+PKW-YW)//2, 0)


    for n in range(YN):
        for c in range(YC):

            for iyh in range(YH):
                for iyw in range(YW):

                    ph_end_const = iyh # iy(h,w) is the bottom-right of pooling window -> All values in pooling window have been computed
                    pw_end_const = iyw

                    ixh_before_stride = iyh+p_st-PKH+1
                    ixw_before_stride = iyw+q_st-PKW+1

                    ixh_beg = int(ixh_before_stride/PSH) # ix(hw) that corresponds to the pooling window
                    ixw_beg = int(ixw_before_stride/PSW)
                    if (ixh_before_stride % PSH != 0) or (ixw_before_stride % PSW != 0): # ix(hw) that corresponds to the window is skipped by pool striding
                        continue

                    if ixh_beg < 0 or ixw_beg <0: # skip with target ix(h,w) < 0
                        continue

                    ph_beg_const = max(PSH*ixh_beg-p_st, 0)-1 # p(h,w)_beg is the index of top left corner of pooling window. If negative, set to zero
                    pw_beg_const = max(PSW*ixw_beg-q_st, 0)-1

                    xh_sweep = PXH if iyh >= YH-PSH else ixh_beg+1 # ix(hw) is sweeped from ix(hw)_beg to x(h,w)_sweep. Normally sweep is 1.
                    xw_sweep = PXW if iyw >= YW-PSW else ixw_beg+1 # But when iy(h,w) is at its edges, need to compute remaining ix(hw) pixels by sweeping

                    ph_end, ph_beg = ph_end_const, ph_beg_const
                    for ixh in range(ixh_beg, xh_sweep):
                        pw_end, pw_beg = pw_end_const, pw_beg_const # move the pooling window back to start of sweep
                        for ixw in range(ixw_beg, xw_sweep):


                            '''
                            Pooling
                            '''
                            result = -math.inf if pool_type == 'max' else 0
                            for ipyh in range(ph_end, ph_beg,-1):
                                for ipyw in range(pw_end, pw_beg,-1):
                                    
                                    if pool_type=='max':
                                        result = max(result, y_arr[n,ipyh,ipyw,c])
                                    else:
                                        result += y_arr[n,ipyh,ipyw,c]

                            count  = (ph_end-ph_beg)*(pw_end-pw_beg)
                            result = result if pool_type=='max' else result/count


                            x_arr[n,ixh,ixw,c] = result
                            pw_beg += PSW # move pooling window by stride
                            pw_end = min(pw_end+PSW, YW-1)
                        ph_beg += PSH # move pooling window by stride
                        ph_end = min(ph_end+PSH, YH-1)


            # for ixh_beg in range(PXH):
            #     for ixw_beg in range(PXW):
                    
            #         ph_end_const = min(PSH*ixh_beg-p_st+PKH-1, YH-1)
            #         pw_end_const = min(PSW*ixw_beg-q_st+PKW-1, YW-1)

            #         ph_beg_const = max(PSH*ixh_beg-p_st, 0)-1
            #         pw_beg_const = max(PSW*ixw_beg-q_st, 0)-1

            #         x_arr[n,ixh_beg,ixw_beg,c] = window_op((ph_beg_const, ph_end_const), (pw_beg_const, pw_end_const), pool_type, y_arr, n, c)

    
    return x_arr

n = 1
c = 1
for _ in range(10):
    for w in Widths:
        for h in Heights:
            for size in pSize:
                for stride in pStride:
                    for m in mode:
                        assert size[0]==size[1], f"pooling size must be square!"
                        x_in = tf.random.uniform(shape=(n, h, w, c), minval=-100, maxval=100, dtype=tf.int32)
                        input_shape = x_in.shape[1:-1] + (c,)
                    
                        x = Input(input_shape, name='input')
                        x1 = MaxPooling2D(size, strides=stride, padding=m)(x)
                        model1 = Model(inputs=[x], outputs=[x1])                        
                        y_ref = model1(x_in).numpy()
                        y_np = myPooling2D("max", x_in.numpy(), size, stride, m)
                    
                        assert y_ref.all() == y_np.all(), f"maxpool error! shape = ({m=}, {size=}, {stride=}, {n=}, {h=}, {w=}, {c=}), {y_ref.shape}, {y_np.shape}, \nx_in[0]=\n{x_in.numpy()[0,:,:,0]}, \ny_ref[0]=\n{y_ref[0,:,:,0]}, \ny_np[0]=\n{y_np[0,:,:,0]}"
                        
                        x2 = AveragePooling2D(size, strides=stride, padding=m)(x)
                        model2 = Model(inputs=[x], outputs=[x2])
                        y_ref = model2(x_in).numpy()
                        y_np = myPooling2D("avg", x_in.numpy(), size, stride, m)
                        assert y_ref.all() == y_np.all(), f"avgpool error! shape = ({m}, {size}, {stride}, {n}, {h}, {w}, {c}), {y_ref.shape}, {y_np.shape}, {x_in.numpy()}, {y_ref}, {y_np}"
                    

In [3]:
# y_ref[0,:,:,0].astype(np.int)

In [4]:
# y_np[0,:,:,0].astype(np.int)

In [5]:
# y_np[0,:,:,0].astype(np.int) - y_ref[0,:,:,0].astype(np.int)